<a href="https://colab.research.google.com/github/shashanksrajak/neural-networks-from-zero/blob/main/hands-on/ch12-custom-models-tensorflow-intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom Models and Training with TensorFlow
This chapter goes deeper into TensorFlow framework and its low level Python APIs. This is useful in two ways - first, we get to learn TensorFlow as it is in its pure form (Keras it a High Level API), second this basic knowledge helps us to take full control of the training flow, we can write custom functions, make our own transformations, optimizers, gradient descent etc etc.

## Tensorflow
- its a computational library, particularly well suited for large ML problems.
- developed by Google and powers lot of its services like photos, speech etc.
- open sourced in 2015 and now its the most widely used deep learning library.

### What does it offer?
1. the core is very similar to NumPy + GPU Support for super computations (PyTorch also has similar capability)
2. distributed computing
3. uses JIT compiler to optimize computations
4. reverse mode autodiff
5. computation graph can be exported to a portable format

In [1]:
import tensorflow as tf

## Using TF like NumPy

### Tensors & Operations
`tf.constant()` works like numpy and creates a tensor

In [4]:
# create a tensor
t = tf.constant(42)
t

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5. ,6.]])
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [6]:
t.shape

TensorShape([2, 3])

In [7]:
t.dtype

tf.float32

In [8]:
t.ndim

2

In [12]:
# indexing
t[0:1, :] # only the 1st row

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[1., 2., 3.]], dtype=float32)>

In [15]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

Mathematical ops

In [16]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [17]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [18]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### Tensors & NumPy

In [19]:
import numpy as np

In [21]:
a = np.array([2, 3, 4], dtype=float)
a

array([2., 3., 4.])

In [26]:
tf.constant(a) # we can pass a numpy array to tensot

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 3., 4.])>

In [27]:
# we can also get a numpy array from a tensor

t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [28]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4.,  9., 16.])>

Numpy can handle data from other libraries and below code shows how easy it is to operate between TensorFlow and NumPy

In [30]:
np.square(t) # this is interesting because we passed a tensor to a numpy method and it just works fine

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### Type Conversion

In [31]:
t1 = tf.constant(42)
t2 = tf.constant(2.)

In [32]:
t1 + t2

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:AddV2] name: 

The above error comes because TF raises error when ops between different dtypes

In [33]:
t2 + tf.cast(t1, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=44.0>

### Variables

`tf.Tensor` objects are **immutable**
so for a mutable data struct we need to use `tf.Variable`

In [34]:
v = tf.Variable([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

Direct assignment does not work, use assignment methods instead.

In [35]:
v[0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

In [36]:
v[0][1]

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [37]:
v[0][1] = 44.0

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [38]:
type(v[0][1])

tensorflow.python.framework.ops.EagerTensor

In [40]:
v[0, 1]

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [41]:
v[0, 1].assign(42.0)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 1., 42.,  3.],
       [ 4.,  5.,  6.]], dtype=float32)>

## Customizing Models & Training Algorithms

### Custom Loss Functions
The goal is to create our own custom Loss function, although the library comes with almost all the loss functions but still if we want to write one then this is how its done.

In [44]:
def huber_loss(y_true, y_pred):
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss = tf.abs(error) - 0.5

  return tf.where(is_small_error, squared_loss, linear_loss)

  """
  return tf.where(is_small_error, squared_loss, linear_loss): This line uses the tf.where operation, which acts like a conditional statement.
  It returns elements from squared_loss where is_small_error is True, and elements from linear_loss where is_small_error is False.
  This effectively combines the squared and linear loss components based on the magnitude of the error, implementing the Huber loss.
  """

In [45]:
# now we can use this loss in training the model
# model.compile(loss=huber_loss, optimizer=nadam)
# model.fit(....)

When we use such custom functions for training the model, and save the model, the custom functions are not going to be saved. The saved model only contains the model arch, weights etc.

So next time we load this model , we should have this custom function in place in this script plus we need to tell TF how to deserialize this custom function.

Alternatively, we can also use a class based approach. Check the book.

In [46]:
# model = tf.keras.models.load_model("custom_model.keras", custom_objects={"huber_loss", huber_loss})

### Custom layers